In [2]:
#Task 1
import pandas as pd
import requests
import json

symbols = input("Enter stock symbols separated by a comma (e.g., AAPL,MSFT): ").strip()

apikey = '' #insert API key here
url = "https://yfapi.net/v6/finance/quote"
headers = {'x-api-key': apikey}
querystring = {"symbols": symbols}

response = requests.get(url, headers=headers, params=querystring)
data = response.json()

for stock in data['quoteResponse']['result']:
    print(f"Stock Ticker: {stock['symbol']}, Company: {stock['shortName']}, Current Market Price: ${stock['regularMarketPrice']}")


Enter stock symbols separated by a comma (e.g., AAPL,MSFT): aapl
Stock Ticker: AAPL, Company: Apple Inc., Current Market Price: $240.36


In [3]:
#Task 2.1
apikey = ''  # insert API key here
url_base = "https://yfapi.net/v11/finance/quoteSummary/"

def fetch_stock_data(symbol, modules):
    headers = {'x-api-key': apikey}
    params = {"modules": ','.join(modules)}
    response = requests.get(f"{url_base}{symbol}", headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        result = data['quoteSummary']['result'][0]
        return {
            "Stock Ticker": symbol,
            "52 Week High": result['summaryDetail']['fiftyTwoWeekHigh']['raw'],
            "52 Week Low": result['summaryDetail']['fiftyTwoWeekLow']['raw'],
            "Return on Assets (ROA)": result['financialData']['returnOnAssets']['raw']
        }
    else:
        print(f"Failed to fetch data for {symbol}: {response.status_code}, {response.text}")
        return None

stock_list = symbols.split(',')
modules = ['summaryDetail', 'financialData']

stock_data = [fetch_stock_data(stock.strip(), modules) for stock in stock_list if stock.strip()]

df = pd.DataFrame(stock_data)

print(df)


  Stock Ticker  52 Week High  52 Week Low  Return on Assets (ROA)
0         aapl         260.1       164.08                 0.22519


In [7]:
# Task 2.2
apikey = ''  # insert API key here
headers = {'x-api-key': apikey}
trending_url = "https://yfapi.net/v1/finance/trending/US"

response = requests.get(trending_url, headers=headers)
trending_stocks = []

if response.status_code == 200:
    stocks = response.json().get('finance', {}).get('result', [{}])[0].get('quotes', [])
    for stock in stocks:
        symbol = stock.get('symbol')
        stock_response = requests.get("https://yfapi.net/v6/finance/quote", headers=headers, params={"symbols": symbol})
        if stock_response.status_code == 200:
            stock_info = stock_response.json().get('quoteResponse', {}).get('result', [{}])[0]
            trending_stocks.append({
                "Stock Ticker": stock_info.get('symbol', 'N/A'),
                "Company Name": stock_info.get('longName', 'N/A'),
                "Current Price": f"${stock_info.get('regularMarketPrice', 'N/A'):,.2f}",
                "52 Week High": f"${stock_info.get('fiftyTwoWeekHigh', 'N/A'):,.2f}",
                "52 Week Low": f"${stock_info.get('fiftyTwoWeekLow', 'N/A'):,.2f}"
            })

# Display the data
trending_df = pd.DataFrame(trending_stocks)
print(trending_df)


   Stock Ticker                          Company Name Current Price  \
0          NVDA                    NVIDIA Corporation       $131.28   
1           CRM                      Salesforce, Inc.       $307.33   
2          SNOW                        Snowflake Inc.       $166.19   
3          IONQ                            IonQ, Inc.        $29.93   
4          BYND                     Beyond Meat, Inc.         $3.56   
5           APP                  AppLovin Corporation       $331.00   
6           NIO                              NIO Inc.         $4.72   
7          MRNA                         Moderna, Inc.        $33.58   
8          TDOC                  Teladoc Health, Inc.        $10.99   
9          MARA                   MARA Holdings, Inc.        $12.45   
10           AI                           C3.ai, Inc.        $26.44   
11         NTNX                         Nutanix, Inc.        $69.35   
12         EBAY                             eBay Inc.        $69.14   
13    